In [2]:
from food.tools import *
from food.paths import *
from food.psql import *
import numpy as np
import torch
from torch.nn import CosineSimilarity
import requests
from food.qdrant import *
cos = CosineSimilarity(dim=1, eps=1e-08)
import pandas as pd

In [2]:
collection_name = 'food'
table = 'foods'
foods = read_sql(table)
# foods = foods.set_index('id')

In [3]:
foods = foods.drop(columns = ['clip'])

In [5]:
foods['text'] = ('the food is ' +
                                  foods['category'] + ' .'
                                  ' It has a little bit of  ' +
                                  foods['description'].str.split(',').apply(lambda l:' '.join(list(reversed(l))))
                                  ).str.lower().str.replace(':','')

In [21]:
foods.to_sql(schema = 'food',name = 'foods_prompted',if_exists = 'append',con=engine,index=False)

In [22]:
foods

,id,description,category,energy,protein,carb,fat,text
0,1097524,"Milk, lactose free, reduced fat (2%)","Milk, reduced fat",50.0,3.35,4.91,1.90,"the food is milk, reduced fat . it has a littl..."
1,1097525,"Milk, lactose free, whole","Milk, whole",60.0,3.28,4.67,3.20,"the food is milk, whole . it has a little bit ..."
2,1097603,"Yogurt, whole milk, baby food",Baby food: yogurt,61.0,3.47,4.66,3.25,the food is baby food yogurt . it has a little...
3,1097700,"Infant formula, NFS","Formula, ready-to-feed",66.0,1.38,7.12,3.56,"the food is formula, ready-to-feed . it has a ..."
4,1099619,Cabbage with ham hocks,Meat mixed dishes,71.0,7.97,4.50,2.49,the food is meat mixed dishes . it has a littl...
...,...,...,...,...,...,...,...,...
7078,1104568,Cheese as ingredient in sandwiches,Not included in a food category,360.0,20.76,4.32,28.93,the food is not included in a food category . ...
7079,1104569,Breakfast meat as ingredient in omelet,Not included in a food category,253.0,20.81,1.61,17.87,the food is not included in a food category . ...
7080,1104570,"Fish fillet, fried as ingredient in sandwiches",Not included in a food category,200.0,18.11,8.18,10.03,the food is not included in a food category . ...
7081,1104571,Breading or batter as ingredient in food,Not included in a food category,201.0,6.32,39.64,1.39,the food is not included in a food category . ...


In [10]:
pd.read_sql('select * from food.foods_prompted where clip is not null limit 5',engine)

,id,description,category,energy,protein,carb,fat,text,clip
0,1098693,Chorizo,Sausages,346.0,19.30,2.63,28.10,the food is sausages . it has a little bit of ...,"[0.03794631, -0.026944248, -0.007641191, -0.01..."
1,1099193,Beef curry,Meat mixed dishes,111.0,7.56,6.46,6.53,the food is meat mixed dishes . it has a littl...,"[-0.0044911825, -0.011347377, 0.023338495, 0.0..."
2,1102682,"Pear, raw",Pears,57.0,0.36,15.23,0.14,the food is pears . it has a little bit of r...,"[0.03659307, -0.022669137, 0.006010248, 0.0017..."
3,1097603,"Yogurt, whole milk, baby food",Baby food: yogurt,61.0,3.47,4.66,3.25,the food is baby food yogurt . it has a little...,"[-0.019953089, -0.04377515, 0.009501303, 0.048..."
4,1097700,"Infant formula, NFS","Formula, ready-to-feed",66.0,1.38,7.12,3.56,"the food is formula, ready-to-feed . it has a ...","[0.026129697, -0.016678246, 0.0027433224, 0.03..."


In [5]:
engine.execute("truncate table food.indexed")